In [1]:
from pyspark import SparkContext, SparkConf, SQLContext, Row
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json
from os import listdir
from os.path import isfile, join
import platform
import pandas as pd
from itertools import chain
import pyspark.sql.functions as func
from datetime import datetime
import numpy as np
import os
from pyspark.sql.types import LongType, DoubleType, StringType, TimestampType, BooleanType

In [2]:
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages com.datastax.spark:spark-cassandra-connector_2.12:3.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1 --conf spark.cassandra.connection.host=127.0.0.1 --conf spark.cassandra.connection.port=9042 --conf spark.cassandra.auth.username=cassandra --conf spark.cassandra.auth.password=cassandra pyspark-shell"

In [3]:
KAFKA_BOOTSTRAP_SERVER = "localhost:9092"

In [4]:
spark = SparkSession \
        .builder \
        .appName("LogsAnalysisWithSpark") \
        .master("local[*]") \
        .getOrCreate()

In [5]:
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

In [6]:
sample = spark.read.json("../../preprocessed/callInfo_data.json")
schema = sample.schema

In [7]:
sample.printSchema()

root
 |-- call: string (nullable = true)
 |-- date: string (nullable = true)
 |-- message: struct (nullable = true)
 |    |-- callInfo: struct (nullable = true)
 |    |    |-- callCorrelator: string (nullable = true)
 |    |    |-- callType: string (nullable = true)
 |    |    |-- distributedInstances: double (nullable = true)
 |    |    |-- endpointRecording: string (nullable = true)
 |    |    |-- joinAudioMuteOverride: double (nullable = true)
 |    |    |-- lockState: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- participants: double (nullable = true)
 |    |    |-- recording: string (nullable = true)
 |    |    |-- streaming: string (nullable = true)
 |    |-- messageId: long (nullable = true)
 |    |-- subscriptionIndex: long (nullable = true)
 |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)



In [8]:
def get_stream_for_topics(topics, spark, kafka_server):
    return spark.readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers", kafka_server)\
    .option("subscribe", ",".join(topics))\
    .load()

In [9]:
streamingInputDF = get_stream_for_topics(["callInfoUpdate11"], spark, KAFKA_BOOTSTRAP_SERVER)

In [10]:
valuesDF = streamingInputDF.selectExpr("CAST(value AS STRING)")

In [11]:
eventsDF = valuesDF.withColumn("event", from_json(valuesDF.value, schema)).select("event")

In [12]:
eventsDF.printSchema()

root
 |-- event: struct (nullable = true)
 |    |-- call: string (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- message: struct (nullable = true)
 |    |    |-- callInfo: struct (nullable = true)
 |    |    |    |-- callCorrelator: string (nullable = true)
 |    |    |    |-- callType: string (nullable = true)
 |    |    |    |-- distributedInstances: double (nullable = true)
 |    |    |    |-- endpointRecording: string (nullable = true)
 |    |    |    |-- joinAudioMuteOverride: double (nullable = true)
 |    |    |    |-- lockState: string (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- participants: double (nullable = true)
 |    |    |    |-- recording: string (nullable = true)
 |    |    |    |-- streaming: string (nullable = true)
 |    |    |-- messageId: long (nullable = true)
 |    |    |-- subscriptionIndex: long (nullable = true)
 |    |    |-- type: string (nullable = true)
 |    |-- type: string (nullable = tr

In [13]:
messagesDF = eventsDF.withColumn("date", eventsDF.event.date)\
                     .withColumn("message", eventsDF.event.message)\
                     .withColumn("call", eventsDF.event.call)\
                     .select("message", "date", "call")

In [14]:
messagesDF.printSchema()

root
 |-- message: struct (nullable = true)
 |    |-- callInfo: struct (nullable = true)
 |    |    |-- callCorrelator: string (nullable = true)
 |    |    |-- callType: string (nullable = true)
 |    |    |-- distributedInstances: double (nullable = true)
 |    |    |-- endpointRecording: string (nullable = true)
 |    |    |-- joinAudioMuteOverride: double (nullable = true)
 |    |    |-- lockState: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- participants: double (nullable = true)
 |    |    |-- recording: string (nullable = true)
 |    |    |-- streaming: string (nullable = true)
 |    |-- messageId: long (nullable = true)
 |    |-- subscriptionIndex: long (nullable = true)
 |    |-- type: string (nullable = true)
 |-- date: string (nullable = true)
 |-- call: string (nullable = true)



In [15]:
preprocessedDF = messagesDF.select("date", "call", messagesDF.message.callInfo.alias("info"))

In [16]:
preprocessedDF.printSchema()

root
 |-- date: string (nullable = true)
 |-- call: string (nullable = true)
 |-- info: struct (nullable = true)
 |    |-- callCorrelator: string (nullable = true)
 |    |-- callType: string (nullable = true)
 |    |-- distributedInstances: double (nullable = true)
 |    |-- endpointRecording: string (nullable = true)
 |    |-- joinAudioMuteOverride: double (nullable = true)
 |    |-- lockState: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- participants: double (nullable = true)
 |    |-- recording: string (nullable = true)
 |    |-- streaming: string (nullable = true)



In [17]:
info = preprocessedDF.info

In [18]:
finalDF = preprocessedDF.select(preprocessedDF.call,
                          info.callType.alias("callType"),
                      info.distributedInstances.alias("distributedInstances"),
                      info.endpointRecording.alias("endpointRecording"),
                      info.lockState.alias("lockState"),
                      info.participants.alias("participants"),
                      info.recording.alias("recording"),
                      info.streaming.alias("streaming"),
                      info.joinAudioMuteOverride.alias("joinAudioMute"),
                      preprocessedDF.date
                     )

In [19]:
# Czas rzeczywisty -> current_time
# Czas od rozpoczęcia spotkania -> time_diff
# Spotkanie nagrywane -> recording
# Spotkanie streamowane -> streaming
# Spotkanie zablokowane -> locked
# Spotkanie adHoc -> adhoc
# Spotkanie
# Aktualna liczba uczestników -> current_participants
# Średnia liczba uczestników w danym spotkaniu -> mean_participants
# Maksymalna liczba uczestników danego spotkania -> max_participants

In [20]:
groupedDF = finalDF.groupBy("call")\
                       .agg(func.sort_array(func.collect_list(finalDF.date)).alias("dateArray"), 
                        func.collect_list(finalDF.recording).alias("recordingArray"),
                        func.collect_list(finalDF.streaming).alias("streamingArray"),
                        func.collect_list(finalDF.lockState).alias("lockStateArray"),
                        reverse(func.collect_list(finalDF.callType)).getItem(0).alias("callType"),
                        reverse(func.collect_list(finalDF.participants)).getItem(0).cast(LongType()).alias("current_participants"),
                        func.max(finalDF.participants).cast(LongType()).alias("max_participants"),
                        func.mean(finalDF.participants).alias("mean_participants"))

In [21]:
groupedDF.printSchema()

root
 |-- call: string (nullable = true)
 |-- dateArray: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- recordingArray: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- streamingArray: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- lockStateArray: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- callType: string (nullable = true)
 |-- current_participants: long (nullable = true)
 |-- max_participants: long (nullable = true)
 |-- mean_participants: double (nullable = true)



In [22]:
def find_diff(dates):
    pattern = '%Y-%m-%dT%H:%M:%S.%f'
    start_date = datetime.strptime(dates[0], pattern)
    end_date = datetime.strptime(dates[-1], pattern)
    return int((end_date - start_date).total_seconds())
    
find_diff_udf = udf(lambda x: find_diff(x), LongType())

In [23]:
def get_last_date(dates):
    pattern = '%Y-%m-%dT%H:%M:%S.%f'
    date = datetime.strptime(dates[-1], pattern)
    return date

get_last_date_udf = udf(lambda x: get_last_date(x), TimestampType())

In [24]:
def get_last_nonempty_value(values):
    nonempty_values = [i for i in values if i]
    return nonempty_values[-1] if nonempty_values else None

In [25]:
def get_if_active(values):
    state = get_last_nonempty_value(values)
    return state == "active"    
    
get_if_active_udf = udf(lambda x: get_if_active(x), BooleanType())

In [26]:
def get_if_locked(values):
    state = get_last_nonempty_value(values)
    return state == "locked"

get_if_locked_udf = udf(lambda x: get_if_locked(x), BooleanType())

In [27]:
def get_if_type(current_type, expected_type):
    return current_type == expected_type

get_if_adhoc_udf = udf(lambda x: get_if_type(x, "adHoc"), BooleanType())
get_if_lync_udf = udf(lambda x: get_if_type(x, "lyncConferencing"), BooleanType())
get_if_forwarding_udf = udf(lambda x: get_if_type(x, "forwarding"), BooleanType())
get_if_cospace_udf = udf(lambda x: get_if_type(x, "coSpace"), BooleanType())

In [28]:
concat_udf = udf(lambda cols: "".join([x if x is not None else "*" for x in cols]), StringType())

In [29]:
finalDF = groupedDF\
                .withColumn("datetime", get_last_date_udf(groupedDF.dateArray))\
                    .withColumn("time_diff", find_diff_udf(groupedDF.dateArray))\
                    .withColumn("call_id", groupedDF.call)\
                    .withColumn("recording", get_if_active_udf(groupedDF.recordingArray))\
                    .withColumn("streaming", get_if_active_udf(groupedDF.streamingArray))\
                    .withColumn("locked", get_if_locked_udf(groupedDF.lockStateArray))\
                    .withColumn("cospace", get_if_cospace_udf(groupedDF.callType))\
                    .withColumn("adhoc", get_if_adhoc_udf(groupedDF.callType))\
                    .withColumn("lync_conferencing", get_if_lync_udf(groupedDF.callType))\
                    .withColumn("forwarding", get_if_forwarding_udf(groupedDF.callType))\
                    .select("datetime", "time_diff", "call_id", "recording", "streaming", 
                            "locked", "cospace", "adhoc", "lync_conferencing", "forwarding",
                            "current_participants", "mean_participants", "max_participants")

In [30]:
finalDF.printSchema()

root
 |-- datetime: timestamp (nullable = true)
 |-- time_diff: long (nullable = true)
 |-- call_id: string (nullable = true)
 |-- recording: boolean (nullable = true)
 |-- streaming: boolean (nullable = true)
 |-- locked: boolean (nullable = true)
 |-- cospace: boolean (nullable = true)
 |-- adhoc: boolean (nullable = true)
 |-- lync_conferencing: boolean (nullable = true)
 |-- forwarding: boolean (nullable = true)
 |-- current_participants: long (nullable = true)
 |-- mean_participants: double (nullable = true)
 |-- max_participants: long (nullable = true)



In [31]:
testDF = finalDF.withColumn("id", concat_udf(func.array(finalDF.call_id, finalDF.datetime)))\
    .withColumn("hour", hour(finalDF.datetime))\
    .withColumn("week_day_number", date_format(finalDF.datetime, 'u').cast(IntegerType()))

In [32]:
testDF.printSchema()

root
 |-- datetime: timestamp (nullable = true)
 |-- time_diff: long (nullable = true)
 |-- call_id: string (nullable = true)
 |-- recording: boolean (nullable = true)
 |-- streaming: boolean (nullable = true)
 |-- locked: boolean (nullable = true)
 |-- cospace: boolean (nullable = true)
 |-- adhoc: boolean (nullable = true)
 |-- lync_conferencing: boolean (nullable = true)
 |-- forwarding: boolean (nullable = true)
 |-- current_participants: long (nullable = true)
 |-- mean_participants: double (nullable = true)
 |-- max_participants: long (nullable = true)
 |-- id: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- week_day_number: integer (nullable = true)



In [33]:
def writeToCassandra(writeDF, epochId):
     writeDF.write \
        .format("org.apache.spark.sql.cassandra") \
        .options(table="test", keyspace="engineering")\
        .mode("append") \
        .save()

In [34]:
writer = testDF\
        .writeStream\
        .outputMode("complete")\
        .foreachBatch(writeToCassandra)

In [35]:
query = writer.start()
query.awaitTermination()

KeyboardInterrupt: 